### `singleton methods`

In [6]:
foo = Object.new

# can define a method directly on that object...
def foo.bar
  "bar"
end

:bar

### `defining methods for many instances of a class`

In [9]:
class Foo
  def bar
    "bar"
  end
end

:bar

In Ruby, we take the everything is an object thing pretty seriously. So naturally a Class is an Object. And we know that objects are instances of a class. So Foo (a Class) is an instance of some class. And that class is Class. I know, it's weird.

In [14]:
Foo.is_a? Class

true

So in fact we can define a class dynamically using Ruby like this. It's actually equivalent to the code we had above.

In [15]:
Foo = Class.new do
  def bar
    "bar"
  end
end

(irb): warning: already initialized constant Object::Foo


#<Class:0x00007fe7809771f0>::Foo

In summary a Class is an object that has the power to spawn new objects. Nontheless they are objects themselves.

### `a Class is an object`

We interact with objects by sending messages. Since we know a Class is an object, we therefore send it messages.

### `how to send methods to a Class object`

Same syntax as how we've been sending messages to non-class objects (instances). Note: there are several ways to define a class method (define how a Class should respond to a specific message). You will see #3 being used a lot in the industry.

In [40]:
class Foo
  # 1
  def self.add_one(x)
    x + 1
  end
  
  # 2
  def Foo.add_two(x)
    x + 2
  end
  
  # 3
  class << self
    def add_three(x)
      x + 3
    end
  end
end

:add_three

In [30]:
p Foo.respond_to? :add_one
p Foo.send :add_one, 1

true
2


2

As an aside, how does #3 work. What is that weird syntax? Forget about the `self` keyword for one bit. And just look to whatever is on the right side of `<<`. We are essentially defining singleton methods on the object that is to the right of `<<`.

In [38]:
obj = Object.new
class << obj
  def yer
    'yerr!!!'
  end
end

:yer

In [39]:
obj.yer

"yerr!!!"

In [41]:
class Foo
  class << self
    def add_three(x)
      x + 3
    end
  end
end

:add_three

So if we have `class << self`, the `self` object in this context resolves into the enclosing class. Therefore, these two are equivalent.

```rb
class Foo
  class << self
    def add_three(x)
      x + 3
    end
  end
end

class Foo
  class << Foo
    def add_three(x)
      x + 3
    end
  end
end
```

### `are attr methods class methods?`

You betcha! The `attr_accessor` method is defined in a Class called Module.

In [61]:
class Foo
  '''
  note that there is a default receiver for each method call
  self.attr_accessor(:bar)
  ^ the self in this case is the enclosing class so Foo
  '''
  attr_accessor(:bar)
  
  p self.method(:attr_accessor)
  
  p Module.methods.include? :attr_accessor
  
end

#<Method: #<Class:#<Class:0x00007fe7809771f0>::Foo>(Module)#attr_accessor(*)>
true


true

In [60]:
Foo.class.superclass

Module

### `what the heck?`

Don't worry it's dead simple. Let's start slow and distinguish what are called class methods vs instance methods in Ruby.

Both are principally methods. And a method is just a message you send to the target object. A class method is simply a message you send to a Class object. And an instance method is simply a message you send to a non-class (instance) object.

When defining a class and writing methods on that class, you are going to specify whether the method is intended for instances of that class or that Class object itself. Let's go back to the drawing board.

In [ ]:
class Foo
  '''
  attr_accessor
  ^ is a class method. How do I know? Look at the default receiver: self, which translates to the class Foo
  
  attr_accessor === self.attr_accessor === Foo.attr_accessor
  
  But where is attr_accessor coming from? We are not defining it in Foo.
  
  Hmm... back the drawing board!
  '''
  
  p self
  
  attr_accessor(:bar)
  p self.method(:attr_accessor)
  p Module.methods.include? :attr_accessor
  
end

### `in Ruby you can't call an instance method with the Class object as the receiver`

In [69]:
class Yogi
  # here we are telling Ruby interpreter that :bera is a message that instances of Yogi can respond to
  def bera
  end
end

Yogi.respond_to? :bera

false

### `in Ruby you can't call a Class method with the instance as the receiver`

In [71]:
class Bazel
  # that :bera is a message that the Bazel (Class) object can respond to
  class << self
    def compile
    end
  end
end

Bazel.new.respond_to? :compile

false

> All that to tell you that the receiver of a method (message) is really important to understanding whether something is a class method or an instance method.

### `classes as objects and message receivers`

Clases are special objects: they're the only kind of object that has the power to spawn new objects (instances). Nonetheless they're objects. To understand where classes get their methods, think about where objects in general get their methods.
  

- From their class
    - X for our example
    
    
- From their own singleton methods `(def obj.talk)`
    - X
  
  
- From the superclass and earlier ancestors of their class
    - should investigate this more...

In [72]:
class Foo
  '''
  attr_accessor
  ^ is a class method. It is a message we can send to a Class object. Not an instance of a class.
  
  Let us look into the parent of the class Foo or the great grandparent
  
  Foo.class 
  Foo.class.superclass
  '''
  
  attr_accessor(:bar)
  
end

[:bar, :bar=]

In [118]:
Foo.class.superclass # and there is our answer. Module is an Class object that defines the attr_accessor class method

Module

In [119]:
Module.methods.include? :attr_accessor

true

### `VOILA!`

So knowing what we do know now. We can define an instance method on the Class/Module object. Remember, other classes (the ones we write in our applications) are all instances of Class/Module. Therefore the instance methods on Class/Module are essentially class methods to our custom Class objects.

In [7]:
Module.methods.include? :new

true

> Snippet from reference: The class Class has both a class-method version of new and an instance method version; the former is called when you write Class.new and the latter when you write Foo.new. A class method is any method that ges called on a CLass object Sharply, a class method is defined, not just called, directly on a Class object. A class object like Foo has its own methods, its own state, and its own identity. It doesn't share these things with instances of itself. Sending a message to Foo isn't the same thing as sending a message to Foo.new.

### `what is ::`

`::` is basically a namespace resolution operator. It allows you to access items in modules or classses. The order doens't matter. You can nest a class inside a module. A module inside a class. Whatever.

Constants in Ruby are nested like files and directories in filesystem. So, constants are uniquely identified by their paths.

```
::Rails::Engine #is an absolute path to the constant.
# like /Rails/Engine in FS.

Rails::Engine #is a path relative to the current tree level.
# like ./Rails/Engine in FS.
```

In [9]:
module Make
  class Claim
  end
end

Make::Claim

#<Class:0x00007fa2341cf308>::Make::Claim

Here is the best way to learn what they are for...

In [9]:
COUNT = 1 
module Frye
  COUNT = 0 # local
  p ::COUNT # seek outside
  p COUNT
end

"done"

(irb): warning: already initialized constant Object::COUNT
(irb):2: warning: already initialized constant #<Class:0x00007fb157105648>::Frye::COUNT
(irb):2: warning: previous definition of COUNT was here


1
0


"done"

Array

In [2]:
DRAGON = 4
module Foo
  DRAGON = 4
  p ::DRAGON
  p DRAGON
end

(irb): warning: already initialized constant Object::DRAGON
(irb):2: warning: already initialized constant #<Class:0x00007fe9b18f1680>::Foo::DRAGON
(irb):2: warning: previous definition of DRAGON was here


NameError: uninitialized constant DRAGON

In [ ]:
### `module`

modules can either be included or extended w/ `include` or `extend`.

To include a module means to treat the methods that are included as instance methods.

To extend a module means to treat the extended methods as class methods.

In [ ]:
module Foo
  def foo
    puts "foo"
  end
end

class Baz
  include Foo
end

class Bar
  extend Foo
end

### `modules are are adjectives`

In [ ]:
module Driveable # modules are also called mixins
end

module Stoppable
end

class Car # now a driveable car
  include Driveable
  include Stoppable
end

In [ ]:
car = Car.new

puts car.is_a? Driveable
puts car.is_a? Stoppable
puts car.is_a? Car
puts car.is_a? Object

In [ ]:
### `modules`

Unlike interfaces can contain state. Like abstract classes, can contains state.

Well they don't exactly contain state and neither do regular classes. Classes/modules contain instance methods - not state. State belongs to the object and get created when an instance method that sets state (setter) is run on the object.

Methods from a module that refer to an instance variable get mixed into class as instance methods. It's only when those instance methods are called on an objet that the instance variables are created - on that object. The instance variables don't belong to the module at all; they belong to the instances of the classes that mix the module.

In [ ]:
module AcceptsComments
  def comments
    @comments ||= []
  end
  
  def add_comment(comment)
    comments.push(comment)
  end
end

In [ ]:
class Video
  include AcceptsComments
end

In [ ]:
v = Video.new
v.add_comment("Yo")
v.add_comment("what")
v.comments

In [ ]:
### `modules should not have an initialize method`

Initialize method from the concrete class will override the initialize method from the mixin.
    
When you mix a module into the class, Ruby will look for methods (both mixins and superclass) by using the ancestors class method. It will return an array with all of the class' mixins and superclasses in the order they will be searched.

In [ ]:
class MySuperClass
  def foo
  end
end

module MyModule
  def foo
  end
end

class MyClass < MySuperClass
  include MyModule
  def foo 
  end
end

In [ ]:
p MyClass.ancestors

In [ ]:
class MySuperClass
  def foo
    puts "MySuperClass"    
  end
end

module MyModule
  def foo
    puts "MyModule"
  end
end

class MyClass < MySuperClass
  include MyModule
  def foo
    super
    puts "MyClass"    
  end
end

In [ ]:
MyClass.new.foo

In [ ]:
### `top level`

The default receiver is `self`. As you can see there are many methods which are available to self in the top-level execution context.

We can mixin methods at different levels, including the top-level.

In [ ]:
module Bespoke
  def suit_up
    p "Yes, sir"
  end
end

include Bespoke
suit_up

self.methods.include? :suit_up

p self

In [ ]:
### `every method call has a receiver`

In [ ]:
# even this has a receiver (implicitly a self receiver)
puts "Hello"

self.puts "Hello"

### `respond_to? / send`

In [ ]:
foo = Object.new

def foo.bar
    "bar"
end

In [ ]:
p foo.respond_to? :bar
p foo.respond_to? "bar"

p foo.send("bar")
p foo.send(:bar)

### `defined?`

In [3]:
class Skirt
end

defined? Skirt

"constant"

In [4]:
module Yer
end

defined? Yer

"constant"

In [1]:
def baz
end

defined? baz

"method"

In [2]:
abra = 1

defined? abra

"local-variable"

In [5]:
defined? self

"self"